In [ ]:
# # !pip install numpy
# # !pip install Pillow
# # !pip install requests
# !pip install pandas
# !pip install tensorflow[and-cuda]
# !pip list | grep tensor

In [ ]:
### Libraries ###
import os, sys, re, time, json, traceback, logging, datetime, gc, shutil, math, base64, pickle
from datetime import datetime
from pathlib import Path
import numpy as np
# import torch
# from numba import cuda
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Image Libraries ###
# import tensorflow as tf

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# if tf.test.gpu_device_name():
#     print('GPU found')
# else:
#     print("No GPU found")
import cv2
import moviepy.editor
from PIL import Image
from deepface import DeepFace
from deepface.detectors import FaceDetector
from sklearn.cluster import KMeans, DBSCAN

In [ ]:
# print(torch.cuda.current_device())
# print(torch.cuda.is_available())
# print(torch.version.cuda)
# print(torch.backends.cudnn.version())
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
### Path of Files ###
input_video_filename_wo_ext = '1_min_video'
input_video_filename = 'Videos_Examples/'+ input_video_filename_wo_ext +'.mp4' # The video files should be on mp4 format
output_video_path = 'Videos_Output/'

In [ ]:
# Create a new directory of output of videos
if not os.path.exists(output_video_path):
    os.makedirs(output_video_path)

# Create a new directory for processed audio if it does not exist
dir_output_audio = output_video_path + input_video_filename_wo_ext +'/Audio/'
if not os.path.exists(dir_output_audio):
    os.makedirs(dir_output_audio)
dir_output_audio_step1 = output_video_path + input_video_filename_wo_ext +'/Audio/Step1_ProcessAudio/'
if not os.path.exists(dir_output_audio_step1):
    os.makedirs(dir_output_audio_step1)
dir_output_audio_step2 = output_video_path + input_video_filename_wo_ext +'/Audio/Step2_Spleeter/'
if not os.path.exists(dir_output_audio_step2):
    os.makedirs(dir_output_audio_step2)
dir_output_audio_step3 = output_video_path + input_video_filename_wo_ext +'/Audio/Step3_Segments/'
if not os.path.exists(dir_output_audio_step3):
    os.makedirs(dir_output_audio_step3)
dir_output_audio_step4 = output_video_path + input_video_filename_wo_ext +'/Audio/Step4_Embeddings/'
if not os.path.exists(dir_output_audio_step4):
    os.makedirs(dir_output_audio_step4)
dir_output_audio_step5 = output_video_path + input_video_filename_wo_ext +'/Audio/Step5_Clusters/'
if not os.path.exists(dir_output_audio_step5):
    os.makedirs(dir_output_audio_step5)
    
# Create a new directory for processed images if it does not exist
dir_output_images = output_video_path + input_video_filename_wo_ext +'/Images/'
if not os.path.exists(dir_output_images):
    os.makedirs(dir_output_images)
dir_output_images_step1 = output_video_path + input_video_filename_wo_ext +'/Images/Step1_Images/'
if not os.path.exists(dir_output_images_step1):
    os.makedirs(dir_output_images_step1)
dir_output_images_step2 = output_video_path + input_video_filename_wo_ext +'/Images/Step2_Faces/'
if not os.path.exists(dir_output_images_step2):
    os.makedirs(dir_output_images_step2)
dir_output_images_step3 = output_video_path + input_video_filename_wo_ext +'/Images/Step3_Embeddings/'
if not os.path.exists(dir_output_images_step3):
    os.makedirs(dir_output_images_step3)
dir_output_images_step4 = output_video_path + input_video_filename_wo_ext +'/Images/Step4_Clusters/'
if not os.path.exists(dir_output_images_step4):
    os.makedirs(dir_output_images_step4)

# Create a new directory for processed text if it does not exist
dir_output_text = output_video_path + input_video_filename_wo_ext +'/Text/'
if not os.path.exists(dir_output_text):
    os.makedirs(dir_output_text)
dir_output_text_seg = output_video_path + input_video_filename_wo_ext +'/Text/Segments/'
if not os.path.exists(dir_output_text_seg):
    os.makedirs(dir_output_text_seg)

In [ ]:
#####################################################
# Video to Image
#####################################################

In [ ]:
%%timeit
### Extract Frames from Videos ###
fields = input_video_filename.split('.mp4')

input_movie = cv2.VideoCapture(input_video_filename)
total_frames = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(input_movie.get(cv2.CAP_PROP_FPS))
video_duration_sec = int(total_frames/fps)
duration = input_movie.get(cv2.CAP_PROP_POS_MSEC)

video = moviepy.editor.VideoFileClip(input_video_filename)
video_duration = int(video.duration)

# Initialize variables
frame_number = 0
frame_counter = 0
CONST_FPS_KEEP = 1
constant_incr = int(fps/CONST_FPS_KEEP)

while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_counter += 1

    # Quit when the input video file ends
    if not ret:
        break

    cv2.imwrite(dir_output_images_step1 + "frame_%d.jpg" % (frame_counter), frame)
    frame_number += constant_incr
    input_movie.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

# print('Stored frame counter:', frame_counter)
# print('Total frames:', total_frames)
# print('video_duration:', video_duration, '\n')

# input_movie.release()
# cv2.destroyAllWindows()
input_movie.release()
cv2.destroyAllWindows()
print('Finish.')

In [ ]:
detector_name = 'mtcnn'
detector = FaceDetector.build_model(detector_name) # set opencv, ssd, dlib, mtcnn or retinaface

In [ ]:
# img = cv2.imread(dir_output_images_step1+'frame_1.jpg')
# faces_obj = FaceDetector.detect_faces(detector, detector_name, img)

In [ ]:
# detected_and_aligned_face = DeepFace.extract_faces(dir_output_images_step1+'frame_1.jpg', detector_backend = 'retinaface')

In [ ]:
%%time
### Recognize Faces ###
# Face Detectors
# detector_name = 'mtcnn'
# detector = FaceDetector.build_model(detector_name) # set opencv, ssd, dlib, mtcnn or retinaface

i = 0
sorted_files = sorted(os.listdir(dir_output_images_step1))
for filename in sorted_files:
    
    s_e_p = filename.split('_')[0]

    img = cv2.imread(dir_output_images_step1+filename)
    faces_obj = FaceDetector.detect_faces(detector, detector_name, img)
    
    
    for detected_face, region, score in faces_obj:
        height, width = len(detected_face), len(detected_face[0])
        
        # Export facial image only if image size is between [48*48, 4096*4096] (pixels)
        # (Face++ documentation: https://console.faceplusplus.com/documents)
        if height >= 48 and width >= 48:
            cv2.imwrite(dir_output_images_step2+'face_%d.jpg' % (i), detected_face)
            #print(dir_output_images_step2+'face_%d.jpg' % (i))
            i += 1

cv2.destroyAllWindows()
# detector = None
# torch.cuda.empty_cache()
# gc.collect()
# device = cuda.get_current_device()
# device.reset()
print('Finish.')

In [ ]:
%%time
### Create Face Embeddings ###
# if not os.path.exists(dir_output_images_step3 + 'embeddings.pkl'): # Create embeddings
embeddings = []
embeddings_filename = []

for filename in sorted(os.listdir(dir_output_images_step2)):
    if '.jpg' in filename:
        embeddings.append(DeepFace.represent(dir_output_images_step2+filename, enforce_detection=False)[0]['embedding'])
        embeddings_filename.append(filename)

# Store pickle/embeddings
embeddings_all = {'embeddings': embeddings, 'filename': embeddings_filename}

f_pickle = open(dir_output_images_step3 + 'embeddings.pkl', "wb")
pickle.dump(embeddings_all, f_pickle)
f_pickle.close()
# else: # Read pickle/embeddings
#     f_pickle = open(dir_output_images_step3 + 'embeddings.pkl', 'rb')
#     embeddings_all = pickle.load(f_pickle)
#     f_pickle.close()

print('Finish.')

In [ ]:
f_pickle = open(dir_output_images_step3 + 'embeddings.pkl', 'rb')
embeddings_all = pickle.load(f_pickle)

In [ ]:
%%timeit
### Cluster Face Embeddings ###
# Delete all clusters folders
for filename in sorted(os.listdir(dir_output_images_step4)):
    if 'cluster' in filename:
        shutil.rmtree(dir_output_images_step4+filename)

n_clusters = 7
clf = KMeans(n_clusters=n_clusters, n_init=100, max_iter=600, random_state=0)
# if kmeans use number of speakers from voice
# clf = DBSCAN(n_jobs=-1, eps=0.11, metric='cosine')
clf.fit(embeddings_all['embeddings'])
preds = clf.labels_

for i in range(0, len(preds)):
    cluster_dir = 'cluster ' + str(preds[i]) + '/'

    if not os.path.exists(dir_output_images_step4 + cluster_dir):
        os.makedirs(dir_output_images_step4 + cluster_dir)
    shutil.copyfile(dir_output_images_step2 + embeddings_all['filename'][i], dir_output_images_step4 + cluster_dir + '/' + embeddings_all['filename'][i])

# torch.cuda.empty_cache()
# gc.collect()
# device = cuda.get_current_device()
# device.reset()
print('Finish.')